In [2]:
!pip install fuzzywuzzy

In [5]:
import fuzzywuzzy
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import pandas as pd

In [69]:
pd.set_option('max_columns', 150)
pd.set_option('max_rows', 1100)

In [11]:
df = pd.read_csv('cleaned_water_data.csv')

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,5,6,7,9,11,12,16,17,18,19,20,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# need to get a list of the main well types from Brian.  Can create a "choice list" in fuzzy wuzzy and apply those all rows
#top ones I see ['Borehole', 'Protected Borehole', 'Tube Well', 'Protected dug well', 'Pump on hand-dug well','Borehole with submersible pump', 'Stand Post', 'spring river lake stream']

In [19]:
df.water_source.value_counts()

Borehole                                                   19558
Protected Borehole                                         16831
Tube Well                                                  16574
Protected dug well                                         13996
Pump on hand-dug well                                      12173
Unknown                                                    11071
Borehole fitted with submersible pump                       9997
Protected Well (no pump)                                    8314
Dug Well                                                    8260
Manual pump on hand-dug well                                7980
gravity river/stream                                        7691
Standpipe or Tapstand                                       7410
Shallow well                                                6367
Protected Spring                                            5438
Spring                                                      5422
Stand Post               

In [44]:
choices= ['Borehole', 'Protected Borehole', 
          'Tube Well', 'Protected dug well', 'Pump on hand-dug well','Borehole with submersible pump',
          'Stand Post', 'spring river lake stream pond', 'municipal water supply', 'tap', 'rain', 'tank', 
          '__MISSING__']

In [23]:
def fuzzymatch(x, choices, scorer, cutoff):
    return process.extractOne(
    x, choices=choices, scorer=scorer, score_cutoff= cutoff)

In [130]:
def fuzzymatch2(x, choices, scorer, cutoff):
    results = process.extractOne(
    x, choices=choices, scorer=scorer, score_cutoff= cutoff)
    if results is None:
        return "no match"
    else:
        return results[0]

In [24]:
uganda = df[(df.country_name == 'Uganda')]

In [37]:
uganda.water_source.fillna('__MISSING__', inplace=True)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [109]:
uganda.water_source.value_counts()
#uganda.loc[:,'water_source']

__MISSING__                                      99313
Borehole                                          4467
Unknown                                           2644
Protected Spring                                  2604
Boreholes                                         1643
Protected Deep Borehole                            766
Unprotected Well                                   349
Protected Shallow Well                             250
Unprotected Spring                                 248
rainwater harvesting                               221
Protected Dug Well                                 198
shallow_well                                       158
Municipal Water Supply                             131
borehole                                           109
Protected Spring Catchment                          73
pond scoop                                          61
Private Tap                                         56
Rainwater Collection                                53
ferro_ceme

In [83]:
uganda['results']= uganda.loc[:,'water_source'].apply(
                                            fuzzymatch,
                                            args=(choices, 
                                                  fuzz.token_set_ratio, 
                                                 70)
)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [131]:
uganda['fuzzy_results']= uganda.loc[:,'water_source'].apply(
                                            fuzzymatch2,
                                            args=(choices, 
                                                  fuzz.token_set_ratio, 
                                                 70)
)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [132]:
uganda.head()

,Row ID,country_name,water_source,water_tech,status_id,management,pay,installer,install_year,status,source,adm1,adm2,wpdx_id,report_date,country_id,activity_id,data_lnk,orig_lnk,photo_lnk,converted,created,updated,lat_deg,lon_deg,Location,Count,fecal_coliform_presence,fecal_coliform_value,subjective_quality,new_report_date,new_install_year,age_well,age_well_days,status_binary,results,results_tech,fuzzy_results
38495,58112,Uganda,__MISSING__,Non-Mechanized Borehole,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058112,08/06/2012 12:00:00 AM +0000,UG,Kiranganzi,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,1.33007,31.3906,"(1.33007, 31.3906)",1,NaN,NaN,NaN,2012-08-06,NaN,NaN,NaT,1,"(__MISSING__, 100)","(Borehole, 100)",__MISSING__
38496,58113,Uganda,__MISSING__,Non-Mechanized Borehole,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058113,07/05/2012 12:00:00 AM +0000,UG,Njarwe (DWD 37910),http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,1.31209,31.3887,"(1.31209, 31.3887)",1,NaN,NaN,NaN,2012-07-05,NaN,NaN,NaT,1,"(__MISSING__, 100)","(Borehole, 100)",__MISSING__
38497,58114,Uganda,__MISSING__,Non-Mechanized Borehole,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058114,07/13/2012 12:00:00 AM +0000,UG,Ruhunga Primary School (DWD 37905),http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,1.23417,31.2732,"(1.23417, 31.2732)",1,NaN,NaN,NaN,2012-07-13,NaN,NaN,NaT,1,"(__MISSING__, 100)","(Borehole, 100)",__MISSING__
38498,58115,Uganda,__MISSING__,Non-Mechanized Borehole,unknown,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058115,07/24/2012 12:00:00 AM +0000,UG,Amwa Health center,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,2.31964,32.3859,"(2.31964, 32.3859)",1,NaN,NaN,NaN,2012-07-24,NaN,NaN,NaT,1,"(__MISSING__, 100)","(Borehole, 100)",__MISSING__
38499,58116,Uganda,__MISSING__,Non-Mechanized Borehole,yes,NaN,NaN,World Vision,NaN,NaN,World Vision US,NaN,NaN,wpdx-00058116,07/24/2012 12:00:00 AM +0000,UG,Bar,http://www.dropbox.com/s/7paz4ixbh9ri3et/DataS...,NaN,NaN,"#source, #report_date, #status_id, #installer,...",10/26/2015 07:09:34 PM +0000,10/26/2015 07:09:34 PM +0000,2.43701,32.4062,"(2.43701, 32.4062)",1,NaN,NaN,NaN,2012-07-24,NaN,NaN,NaT,0,"(__MISSING__, 100)","(Borehole, 100)",__MISSING__


In [133]:
uganda.fuzzy_results.value_counts()

__MISSING__                      99313
Borehole                          7009
no match                          3613
Protected Borehole                2604
Protected dug well                 800
tap                                138
municipal water supply             131
rain                                33
spring river lake stream pond       26
Name: fuzzy_results, dtype: int64

In [134]:
#uganda.loc[~uganda.results.isin(choices[0:5])]
uganda[['water_source', 'fuzzy_results']].loc[~uganda.water_source.isin(['__MISSING__', 'Boreholes', 'Borehole'])]

,water_source,fuzzy_results
207538,Protected Spring,Protected Borehole
210710,Protected Spring,Protected Borehole
212337,Protected Spring,Protected Borehole
212670,Protected Spring,Protected Borehole
213196,Unknown,no match
213869,Protected Spring,Protected Borehole
214581,Protected Spring,Protected Borehole
214663,Protected Spring,Protected Borehole
214908,Protected Spring,Protected Borehole
214975,Protected Spring,Protected Borehole


In [140]:
#uganda.results.fillna('bad_match')
uganda.groupby(['water_source', 'fuzzy_results']).apply(lambda x: x.values)

water_source                                   fuzzy_results                
 Bottled Water                                 no match                         [[406405, Uganda,  Bottled Water, __MISSING__,...
 Dam                                           no match                         [[405096, Uganda,  Dam, __MISSING__, yes, nan,...
 Lake, pond, or other standing water source    no match                         [[406012, Uganda,  Lake, pond, or other standi...
 River, stream, or other flowing water source  no match                         [[410467, Uganda,  River, stream, or other flo...
 Shifting well                                 no match                         [[415078, Uganda,  Shifting well, __MISSING__,...
Borehole                                       Borehole                         [[404889, Uganda, Borehole, __MISSING__, yes, ...
Boreholes                                      Borehole                         [[160892, Uganda, Boreholes, __MISSING__, no, ...
Dam connected

In [86]:
uganda.results.fillna('bad_match')
uganda.results.isnull().sum()

3613

In [152]:
choices_tech= ['Borehole', 'India Mark MK II', 'Afridev', 'gravity', 'pump',
          'Tube Well', 'stand', 'tap', 'bucket',
           'spring river lake stream pond', 'municipal water supply', 'rain rainwater harvest tank', 
          '__MISSING__', 'Shallow Well', 'dug well']

In [153]:
#uganda.water_tech.value_counts()

In [94]:
uganda.water_tech.fillna('__MISSING__', inplace=True)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [95]:
uganda.water_tech.isnull().sum()

0

In [154]:
uganda['results_tech']= uganda.loc[:,'water_tech'].apply(
                                            fuzzymatch,
                                            args=(choices_tech, 
                                                  fuzz.token_set_ratio, 
                                                 70)
)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [155]:
print("number of uganda water tech categories before fuzzy matching",len(uganda.water_tech.unique()))
print("Number of uganda water tech categories after fuzzy matching", len(uganda.results_tech.unique()))
#fuzzy matching results
uganda.results_tech.value_counts()

number of uganda water tech categories before fuzzy matching 106
Number of uganda water tech categories after fuzzy matching 16


(Borehole, 100)                         26723
(Shallow Well, 100)                     17052
(rain rainwater harvest tank, 100)      15417
(__MISSING__, 100)                      12161
(stand, 100)                             7756
(tap, 100)                               5742
(pump, 100)                              1718
(gravity, 100)                            259
(India Mark MK II, 100)                   137
(dug well, 100)                            98
(rain rainwater harvest tank, 81)          38
(spring river lake stream pond, 100)        1
(Borehole, 73)                              1
(Borehole, 94)                              1
(rain rainwater harvest tank, 75)           1
Name: results_tech, dtype: int64

In [141]:
uganda['fuzzy_tech']=uganda.loc[:,'water_tech'].apply(
                                            fuzzymatch2,
                                            args=(choices_tech, 
                                                  fuzz.token_set_ratio, 
                                                 70)
)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [142]:
#check out errors
uganda[['water_tech', 'fuzzy_tech']].loc[uganda.water_tech.isin(['__MISSING__'])]

,water_tech,fuzzy_tech
38495,Non-Mechanized Borehole,Borehole
38496,Non-Mechanized Borehole,Borehole
38497,Non-Mechanized Borehole,Borehole
38498,Non-Mechanized Borehole,Borehole
38499,Non-Mechanized Borehole,Borehole
38500,Non-Mechanized Borehole,Borehole
38502,Non-Mechanized Borehole,Borehole
38503,Non-Mechanized Borehole,Borehole
38504,Non-Mechanized Borehole,Borehole
38505,Non-Mechanized Borehole,Borehole


In [122]:
#uganda.groupby(['water_tech', 'results_tech']).count()

In [156]:
#uganda[uganda['water_tech'].str.contains('rain')]